<a href="https://colab.research.google.com/github/Itsuki-Hamano123/Quantum_prac/blob/master/gate_qaoa_vqe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My Drive/量子コンピュータ練習/ナレッジ用に解く

/content/drive/My Drive/量子コンピュータ練習/ナレッジ用に解く


In [0]:
!pip install wildqat
!pip install blueqat

#量子ゲートで組み合わせ最適化を行う
参考サイト（https://qiita.com/YuichiroMinato/items/a13ad99162e29de94f01#%E3%81%AF%E3%81%98%E3%82%81%E3%81%AB）

現在の量子ゲートマシンは、NISQ（Noisy Intermediate-Scale Quantum Computer）と言われる「誤り訂正無しエラーあり中規模マシン」が使われています。

量子コンピュータは量子の重ね合わせ状態を利用し演算を行うのですが、ハードの制約上、ノイズを受けたり、計算時間が長すぎると量子状態が壊れてしまい正しい結果を得ることができません。壊れた量子状態を正す、誤り訂正が実装された量子ゲートマシンが理想ですが、まだ研究途中で実用化されるのは、かなり先になりそうです。（2050年くらいという研究者もいるくらい）

しかし、そこで悲観せずに、理想の量子コンピュータが出る前に、量子コンピュータの有用性を見出すために、実験的にプロトタイプとしてNISQが用いられています。

NISQを上手く使うためには、少し工夫が必要になります。それは、NISQでの計算時間を、量子状態が壊れるよりも前に終わるよう、短くする工夫です。

そこで、例えば、１つの数式を分割し、複数回に分けてNISQで計算し、集計と最適化を古典コンピュータで行う、量子・古典ハイブリットアルゴリズムなどが提案されています。
今回は、この量子・古典ハイブリットアルゴリズムの内、QAOAとVQEを使い、コスト関数（イジングモデル）の最適化計算を行ってみます。

**解きたい問題<br>**
$ab=3$を満たす整数$a,b$の組み合わせを探す（iging_3to2.ipynbと同じ）


**解く手順**
1. 問題の定式化
2. QAOAとVQEを使って解を求める

# QAOAとは
QAOAとは、量子アニーリングマシンで動いている量子断熱計算アルゴリズム（SAの量子版）をベースに、量子ゲートで組み合わせ最適化問題の近似解を求めるためのアルゴリズムです。





In [0]:
import wildqat as wq
import numpy as np
from blueqat import vqe
a = wq.opt()

qubo1 = np.array([[0,0,-5,-8,0,4],[0,0,-8,-8,0,16],[0,0,0,0,4,0],[0,0,0,0,16,0],[0,0,0,0,0,16],[0,0,0,0,0,0]])
qubo2=np.array([[0,1,0,0,-2,0],[0,0,0,0,-2,0],[0,0,0,1,0,-2],[0,0,0,0,0,-2],[0,0,0,0,3,0],[0,0,0,0,0,3]])

qubo=(0.01*qubo1)+(0.7*qubo2)
print(qubo)

[[ 0.    0.7  -0.05 -0.08 -1.4   0.04]
 [ 0.    0.   -0.08 -0.08 -1.4   0.16]
 [ 0.    0.    0.    0.7   0.04 -1.4 ]
 [ 0.    0.    0.    0.    0.16 -1.4 ]
 [ 0.    0.    0.    0.    2.1   0.16]
 [ 0.    0.    0.    0.    0.    2.1 ]]


計算精度の調整はstep数で行います。stepというところを増やしたり減らしたりすることで精度と計算速度を調整できます。問題サイズが大きくなるとstep数を増やすことを推奨します。

In [0]:
#step=25だと3分くらい時間かかった気がする
step = 25
qubo_pauli=wq.pauli(qubo)
result = vqe.Vqe(vqe.QaoaAnsatz(qubo_pauli,step)).run()
print(result.most_common(5))

(((1, 1, 1, 0, 1, 0), 0.14806066716377012), ((1, 0, 1, 1, 0, 1), 0.14806066716376992), ((0, 0, 0, 0, 0, 0), 0.129546916356916), ((1, 1, 0, 0, 1, 0), 0.06518684288587578), ((0, 0, 1, 1, 0, 1), 0.06518684288587578))


In [0]:
#アニーリングで解いた場合
a.qubo=qubo
result=a.run(shots=100,sampler="fast",verbose=True)
wq.counter(result)

0 : [1, 0, 0, 1, 0, 0]
1 : [0, 1, 1, 0, 0, 0]
2 : [0, 1, 0, 1, 0, 0]
3 : [1, 0, 0, 1, 0, 0]
4 : [0, 1, 0, 1, 0, 0]
5 : [1, 0, 1, 1, 0, 1]
6 : [1, 0, 0, 1, 0, 0]
7 : [1, 0, 1, 1, 0, 1]
8 : [1, 0, 1, 1, 0, 1]
9 : [1, 0, 1, 0, 0, 0]
10 : [1, 0, 1, 1, 0, 1]
11 : [1, 0, 1, 0, 0, 0]
12 : [1, 0, 1, 1, 0, 1]
13 : [1, 0, 1, 0, 0, 0]
14 : [0, 1, 1, 0, 0, 0]
15 : [1, 0, 1, 1, 0, 1]
16 : [0, 1, 0, 1, 0, 0]
17 : [1, 0, 1, 1, 0, 1]
18 : [1, 1, 1, 0, 1, 0]
19 : [1, 0, 0, 1, 0, 0]
20 : [1, 0, 0, 1, 0, 0]
21 : [0, 1, 1, 0, 0, 0]
22 : [0, 1, 0, 1, 0, 0]
23 : [0, 1, 0, 1, 0, 0]
24 : [1, 1, 0, 1, 1, 0]
25 : [1, 0, 1, 1, 0, 1]
26 : [1, 1, 1, 0, 1, 0]
27 : [1, 1, 1, 0, 1, 0]
28 : [1, 0, 0, 1, 0, 0]
29 : [0, 1, 0, 1, 0, 0]
30 : [1, 1, 1, 0, 1, 0]
31 : [0, 1, 1, 0, 0, 0]
32 : [1, 0, 0, 1, 0, 0]
33 : [1, 0, 0, 1, 0, 0]
34 : [0, 1, 0, 1, 0, 0]
35 : [0, 1, 1, 0, 0, 0]
36 : [1, 0, 1, 1, 0, 1]
37 : [0, 0, 0, 0, 0, 0]
38 : [1, 0, 0, 1, 0, 0]
39 : [1, 0, 0, 1, 0, 0]
40 : [1, 1, 1, 0, 1, 0]
41 : [0, 1, 1, 0, 0, 0]
42

Counter({'000000': 2,
         '001000': 1,
         '010000': 1,
         '010100': 16,
         '011000': 12,
         '100100': 18,
         '101000': 8,
         '101101': 24,
         '110110': 1,
         '111010': 17})